In [6]:
import json

In [7]:
with open('data/dx.json') as f:
    pars_dx = json.load(f)
    
with open('data/itr.json') as f:
    dat_itr = json.load(f)    


In [8]:
dat_itr

{'Presumptive': {'SSM': 13914911, 'CBNAAT': 1370649, 'Truenat': 2749903},
 'Notification': {'Sector_Public': 1688427,
  'Sector_Private': 733694,
  'Type_New': 2131704,
  'Type_Treated': 226960,
  'Type_PMDT': 63457,
  'Test_SSM': 513050,
  'Test_Xpert': 548981,
  'Test_CXR': 835930,
  'Test_Other': 1037210},
 'Test_Truenat': {'Tested': 3483130,
  'Detected': 529196,
  'TestRif': 490267,
  'DetectedRif': 21659,
  'Tested_Private': 215594,
  'Detected_Private': 53067,
  'DetectedRif_Private': 3735},
 'Test_CBNAAT': {'Tested': 2365739,
  'Detected': 525088,
  'DetectedRif': 42026,
  'Tested_Private': 262160,
  'Detected_Private': 68556,
  'DetectedRif_Private': 6531},
 'Test_LPA': {'Tested': 309719, 'DetectedRif': 27180},
 'Year': 2022,
 'Location': 'India'}

In [9]:
pars_dx

{'sens_ssm': 0.64,
 'spec_ssm': 0.98,
 'sens_xpert': 0.85,
 'sens_xpert_ss-': 0.64,
 'spec_xpert': 0.98,
 'p_ava_xpert_pub': 0.2,
 'p_ava_ssm_pub': 0.8,
 'p_ava_xpert_eng': 0.3,
 'p_loss_sputum': 0.15,
 'p_loss_swab': 0.02,
 'sens_cdx': 0.7,
 'spec_cdx': 0.95,
 'p_csi_pub': 0.483,
 'p_csi_ppm': 0.6,
 'dur_pri': 0.7,
 'dur_pub': 0.5,
 'p_txi_pub': 0.9,
 'p_txi_eng': 0.8,
 'p_txi_pri': 0.8,
 'p_refer_i2u': 0.3}

In [11]:
def find_p(sens, spec, n, x):
    p = x / n - (1 - spec)
    p /= (sens - (1 - spec))
    return p

In [12]:
p_tb = find_p(pars_dx['sens_ssm'], pars_dx['spec_ssm'], dat_itr['Presumptive']['SSM'], dat_itr['Notification']['Test_SSM'])
p_tb

0.027210515627861805

In [13]:
sens_xpert, spec_xpert = pars_dx['sens_xpert'], pars_dx['spec_xpert']

n_xpert = dat_itr['Presumptive']['CBNAAT'] + dat_itr['Presumptive']['Truenat']
x_xpert = n_xpert * p_tb
x_xpert_sn = dat_itr['Notification']['Test_Xpert'] - x_xpert

sens2, spec2 = 0.85, 0.98
find_p(sens2, spec2, n2, x2)

0.13642164494652922

In [7]:
find_p(0.85, 0.98, 1370649 + 2749903, 548981)

0.13642164494652922

In [8]:
513050 / 13914911, 548981 / (1370649 + 2749903)

(0.036870519689274336, 0.13322996530561926)

In [11]:
tp1, fp1 = 13914911 * p1 * 0.64, 13914911 * (1 - p1) * 0.02



In [19]:
tp1 * sens2 / (tp1 * sens2 + fp1 * (1 - spec2))

0.9743861828847827

In [40]:
sens2, spec2 = 0.95, 0.98

p1 = find_p(0.64, 0.98, 13914911, 513050)
tp1, fp1 = 13914911 * p1 * 0.64, 13914911 * (1 - p1) * 0.02

k = 1.705
n3 = n2 - (tp1 + fp1) * k

tp22, fp22 = (n3 * p1 + tp1 * k) * sens2, (n3 * (1 - p1) + fp1 * k) * (1 - spec2)
x22 = tp22 + fp22
x22, x2, x22 / x2, n3

(548790.2970002838, 548981, 0.9996526236796608, 3245801.75)